In [1]:
import re
import numpy as np
from scipy.interpolate import UnivariateSpline
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import seaborn as sns
import kw

In [2]:
# データの読み込み

data = kw.getdf_xlsx()['sheet1']
data.head(2)

,TESTDATE,test_no,capacity,total_head,current,output,shaft_power,theoretical_power,pump_eff
0,2015-09-11,1,0.000,19.7,2.50,0.492,0.394,0.000000,0.000000
1,2015-09-11,2,0.025,19.3,2.61,0.564,0.459,0.078808,17.169572


In [55]:
melted_df = pd.melt(data, id_vars=['TESTDATE', 'test_no'], 
                    value_vars=['capacity', 'total_head', 'current', 'output', 'shaft_power','theoretical_power', 'pump_eff'],
                    var_name='label', value_name='value')

In [56]:
# Group by the TESTDATE, label, and test_no, and calculate the mean of the duplicate values
aggregated_df = melted_df.groupby(['TESTDATE', 'label', 'test_no'], as_index=False).mean()
aggregated_df

,TESTDATE,label,test_no,value
0,2015-01-24,capacity,1,0.000000
1,2015-01-24,capacity,2,0.025000
2,2015-01-24,capacity,3,0.050000
3,2015-01-24,capacity,4,0.120000
4,2015-01-24,capacity,5,0.200000
...,...,...,...,...
4153,2023-07-16,total_head,2,19.269539
4154,2023-07-16,total_head,3,19.105433
4155,2023-07-16,total_head,4,17.804390
4156,2023-07-16,total_head,5,15.197805


In [57]:
# Pivot the aggregated DataFrame into the desired format
pivoted_df = aggregated_df.pivot(index=['TESTDATE', 'label'], columns='test_no', values='value').reset_index()

# Preview the transformed DataFrame
pivoted_df.head()

test_no,TESTDATE,label,1,2,3,4,5,6
0,2015-01-24,capacity,0.000000,0.025000,0.050000,0.120000,0.200000,0.280000
1,2015-01-24,current,2.484496,2.614173,2.770556,3.212381,3.575940,3.844099
2,2015-01-24,output,0.505925,0.574376,0.648188,0.837062,1.009881,1.115897
3,2015-01-24,pump_eff,0.000000,17.887100,29.858007,49.916415,61.299749,52.528006
4,2015-01-24,shaft_power,0.394137,0.440553,0.522389,0.699090,0.810086,0.898032


In [33]:
# Pivot the aggregated DataFrame into the desired format
pivoted_df = aggregated_df.pivot(index=['TESTDATE', 'label'], columns='test_no', values='value').reset_index()

# Preview the transformed DataFrame
pivoted_df.head()

test_no,TESTDATE,label,1,2,3,4,5,6
0,2015-01-24,capacity,0.000000,0.025000,0.050000,0.120000,0.200000,0.280000
1,2015-01-24,current,2.484496,2.614173,2.770556,3.212381,3.575940,3.844099
2,2015-01-24,output,0.505925,0.574376,0.648188,0.837062,1.009881,1.115897
3,2015-01-24,shaft_power,0.394137,0.440553,0.522389,0.699090,0.810086,0.898032
4,2015-01-24,total_head,19.709501,19.298486,19.098968,17.804106,15.201440,10.314543


In [58]:
dict = {'sheet1': pivoted_df}
kw.writedf_xlsx(dict)

In [65]:
# Step 1: Use `melt` to convert sheet2 data to long format
melted_df2 = pivoted_df.melt(id_vars=["TESTDATE", "label"], 
                            value_vars=[1, 2, 3, 4, 5, 6], 
                            var_name="test_no", 
                            value_name="value")

In [66]:
# Ensure the data types are consistent for further processing
melted_df2["test_no"] = melted_df2["test_no"].astype(int)

In [68]:
# Step 2: Use `pivot` to reshape the long data into the format of sheet1
pivoted_df2 = melted_df2.pivot(index=["TESTDATE", "test_no"], 
                                     columns="label", 
                                     values="value").reset_index()
pivoted_df2.head()

label,TESTDATE,test_no,capacity,current,output,pump_eff,shaft_power,theoretical_power,total_head
0,2015-01-24,1,0.000,2.484496,0.505925,0.000000,0.394137,0.000000,19.709501
1,2015-01-24,2,0.025,2.614173,0.574376,17.887100,0.440553,0.078802,19.298486
2,2015-01-24,3,0.050,2.770556,0.648188,29.858007,0.522389,0.155975,19.098968
3,2015-01-24,4,0.120,3.212381,0.837062,49.916415,0.699090,0.348960,17.804106
4,2015-01-24,5,0.200,3.575940,1.009881,61.299749,0.810086,0.496580,15.201440
